# Detección de Idiomas en el Abstract

In [2]:
#Lectura de Librerías
import pandas as pd
import time
#pd.set_option('display.max_columns', None)

In [3]:
#Leyendo los abstracts
df = pd.read_csv(r'Data/abstract_eng.csv',sep=',')

## Remover Stopwords 

In [4]:
import nltk
from nltk import *
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ECSIM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
stop = stopwords.words('english')

In [6]:
#Se remueve el dato nan
df = df[df['Abstract_ing'].isna()==False]

In [7]:
Abstracts = df[['Title','Abstract_ing']].drop_duplicates()
Abstracts.reset_index(inplace=True)
Abstracts["Abstract_ing"] = Abstracts["Abstract_ing"].str.lower().str.split()
#Abstracts.shape

In [8]:
start = time.time()
Abstracts['Clean_Abstract'] = Abstracts[['Abstract_ing']] \
.apply(lambda x: [item for item in x if item not in stop])
end = time.time()
print((end-start)/60)

0.01038124958674113


In [9]:
Abstracts['Clean_Abstract'] = Abstracts['Abstract_ing'] \
.apply(lambda x: ' '.join([word for word in x]))

In [10]:
del(df)

The common way of doing this is to transform the documents into tf-idf vectors, 
then compute the cosine similarity between them. Any textbook on information retrieval (IR) covers this. See esp. Introduction to Information Retrieval, which is free and available online.

Tf-idf (and similar text transformations) are implemented in the Python packages
Gensim and scikit-learn. In the latter package, computing cosine similarities is as easy as


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
start=time.time()
abstracts = Abstracts['Clean_Abstract']
#stopwords_ = [word.decode('utf-8') for word in stopwords.words('english')]
tfidf = TfidfVectorizer().fit_transform(abstracts)
# no need to normalize, since Vectorizer will return normalized tf-idf
end = time.time()
print((end-start)/60)

0.1364099343617757


### Otras posibles Opciones
https://stackoverflow.com/questions/51591510/text-similarity-approaches-do-not-reflect-real-similarity-between-texts
https://stackoverflow.com/questions/8897593/how-to-compute-the-similarity-between-two-text-documents
https://stackoverflow.com/questions/101569/algorithm-to-detect-similar-documents-in-python-script

#Apuntes
#https://stackoverflow.com/questions/25443802/unicode-warning-when-using-nltk-stopwords-with-tfidfvectorizer-of-scikit-learn
https://hdbscan.readthedocs.io/en/latest/performance_and_scalability.html

In [15]:
from sklearn.cluster import KMeans

In [16]:

temp_dict={}


In [19]:
start = time.time()
columnas = []
for i in range(3,20,1):
    if (i==13):
        continue
    nombre = "C_" + str(i)
    columnas.append(nombre)
    kmeans = KMeans(n_clusters=i).fit(tfidf)
    temp_dict[nombre] = kmeans.labels_
end = time.time()
print((end-start)/60)

343.4885528008143


In [27]:
df_category = pd.DataFrame(temp_dict)

In [31]:
df_temp=pd.concat([Abstracts,df_category], axis=1)

In [ ]:
#df_temp = df_temp[['Title','C_10','C_11','C_12','C_13','C_14','C_15','C_16','C_17','C_18','C_19','C_20']]
#df_final = df.merge(df_temp, how='right', on = 'Title')

In [ ]:
#df_final.drop(columns='Unnamed: 0', axis=1, inplace=True)

In [ ]:
#df_final

In [32]:
df_temp.to_csv(r'Data/scopus_cluster2.csv')

In [25]:
kmeans13 = KMeans(n_clusters=13).fit(tfidf)
temp_dict['C_13'] = kmeans13.labels_

In [26]:
temp_dict

{'C_3': array([0, 2, 0, ..., 0, 0, 2]),
 'C_4': array([3, 0, 3, ..., 3, 3, 0]),
 'C_5': array([0, 3, 0, ..., 0, 0, 4]),
 'C_6': array([0, 1, 0, ..., 0, 0, 5]),
 'C_7': array([6, 4, 6, ..., 0, 6, 2]),
 'C_8': array([4, 7, 4, ..., 0, 4, 5]),
 'C_9': array([3, 1, 3, ..., 7, 3, 2]),
 'C_10': array([8, 2, 8, ..., 4, 8, 6]),
 'C_11': array([0, 8, 1, ..., 1, 1, 7]),
 'C_12': array([11,  6, 11, ...,  5, 11,  7]),
 'C_14': array([0, 2, 1, ..., 1, 1, 1]),
 'C_15': array([ 0,  2,  0, ..., 10,  0,  4]),
 'C_16': array([12,  8,  9, ...,  9, 12,  9]),
 'C_17': array([10, 13,  8, ..., 15,  8,  8]),
 'C_18': array([ 2, 13, 12, ...,  9, 12, 12]),
 'C_19': array([14,  6, 15, ..., 15, 15, 15]),
 'C_13': array([12,  9,  7, ...,  7,  7,  7])}